In [1]:
!pip install https://gitlab.com/trungtv/vi_spacy/-/raw/master/packages/vi_core_news_lg-3.6.0/dist/vi_core_news_lg-3.6.0.tar.gz -q
!pip install pyvi qdrant-client sentence-transformers -q
!python -m spacy validate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 MB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.4/917.4 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 95.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 21

In [2]:
import re
import pandas as pd
import json
import pickle
import difflib
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import os
import warnings

# Tắt các cảnh báo không cần thiết
warnings.filterwarnings("ignore", category=FutureWarning, module="huggingface_hub")

# ==============================================================================
# === 1. CẤU HÌNH CÁC MÔ HÌNH VÀ ĐƯỜNG DẪN =======================================
# ==============================================================================

MODEL_CONFIGURATIONS = [
    {
        "name": "halong_finetuned",
        "model_path": "/kaggle/input/embedding-train/model_test"
    },
    {
        "name": "phobert_finetuned",
        "model_path": "/kaggle/input/embedding-train-sup-simcse-vietnamese-phobert-base/sup_SimCSE_VietNamese_phobert_base"
    },
    {
        "name": "halong_base",
        "model_path": "hiieu/halong_embedding"
    },
    {
        "name": "phobert_base",
        "model_path": "VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"
    }
]

INPUT_CSV_PATH = "/kaggle/input/data-uit/train.csv"
OUTPUT_DIR = "/kaggle/working/processed_data_regex"

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Các file output sẽ được lưu tại: {OUTPUT_DIR}")

# ==============================================================================
# === 2. CÁC HÀM HỖ TRỢ (HÀM CHUNKING ĐÃ ĐƯỢC SỬA LỖI) ==========================
# ==============================================================================

def remove_redundant_repeats(text):
    """Xoá các đoạn bị lặp như "Điều 9. Điều 9."""
    return re.sub(r"(Điều\s+\d+\.)\s+\1", r"\1", text)

def split_into_semantic_chunks_regex(text):
    """
    [ĐÃ SỬA LỖI] Chia đoạn theo các tiêu chí rõ nghĩa dựa trên regex.
    Hàm này giờ đây xử lý kết quả từ re.split một cách an toàn hơn.
    """
    text = remove_redundant_repeats(str(text))
    # Pattern vẫn giữ nguyên, với một capturing group cho toàn bộ delimiter
    pattern = r'((?:^|\n)(?:Điều\s+\d+\.|[IVXLCDM]+\.\s|[0-9]+\.\s|[a-z]\)\s|–\s|-\s))'
    
    # Split text, kết quả sẽ xen kẽ giữa text và delimiter
    # Ví dụ: [text_before, delimiter_1, text_after_1, delimiter_2, text_after_2, ...]
    parts = re.split(pattern, text)
    
    result = []
    # Phần text đầu tiên (trước delimiter đầu tiên)
    if parts[0] and parts[0].strip():
        result.append(parts[0].strip())
        
    # Ghép các delimiter với phần text theo sau nó
    # Duyệt qua các cặp (delimiter, text)
    for i in range(1, len(parts), 2):
        # Đảm bảo không truy cập ra ngoài danh sách
        if i + 1 < len(parts):
            # Ghép delimiter (parts[i]) với nội dung của nó (parts[i+1])
            chunk = (parts[i] + parts[i+1]).strip()
            if chunk:
                result.append(chunk)
    
    # Nếu không split được gì, trả về text gốc nếu nó có nội dung
    if not result and text.strip():
        return [text.strip()]

    # Lọc cuối cùng để loại bỏ các chunk quá ngắn hoặc chỉ chứa ký tự gạch nối
    return [c for c in result if len(c.strip()) > 10 and not re.fullmatch(r'[-–]+', c.strip())]


def clean_text(text):
    """Dọn dẹp văn bản cơ bản."""
    cleaned_text = re.sub(r'[^\w\s.,]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

def count_words(text):
    """Đếm số từ trong một chuỗi."""
    return len(text.split())

def strings_similar(s1, s2, threshold=0.95):
    """Kiểm tra sự giống nhau của hai chuỗi."""
    return difflib.SequenceMatcher(None, s1, s2).ratio() > threshold

# ==============================================================================
# === 3. CHUNKING VÀ XỬ LÝ DỮ LIỆU (CHẠY 1 LẦN) ================================
# ==============================================================================

print("\nBắt đầu giai đoạn 1: Chunking và xử lý dữ liệu...")

try:
    df = pd.read_csv(INPUT_CSV_PATH)
    print(f"✔ Đã đọc thành công file: {INPUT_CSV_PATH} ({len(df)} hàng)")
except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file {INPUT_CSV_PATH}. Vui lòng kiểm tra đường dẫn.")
    df = pd.DataFrame()

all_payloads = []
unique_chunks_text = []
test_data = []
global_id_counter = 0

if not df.empty:
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Đang xử lý các context"):
        original_text_context = str(row.get('context', ''))
        if not original_text_context.strip():
            continue

        document_title = str(row.get("document", ""))
        article_info = str(row.get("article", ""))
        question = str(row.get("question", ""))

        # Bước 1: Chia nhỏ bằng regex (hàm đã sửa lỗi)
        initial_chunks = split_into_semantic_chunks_regex(original_text_context)

        # Bước 2: Gộp lại các chunk nhỏ thành các chunk lớn hơn (~200 từ)
        processed_document_chunks = []
        if initial_chunks:
            current_chunk_buffer = ""
            for chunk in initial_chunks:
                # Nếu thêm chunk mới vào sẽ vượt quá 200 từ, thì lưu buffer cũ lại
                if count_words(current_chunk_buffer + " " + chunk) > 200 and current_chunk_buffer:
                    processed_document_chunks.append(current_chunk_buffer)
                    current_chunk_buffer = chunk
                else:
                    # Nếu buffer rỗng thì bắt đầu, nếu không thì nối thêm
                    if not current_chunk_buffer:
                        current_chunk_buffer = chunk
                    else:
                        current_chunk_buffer += " " + chunk
            # Lưu lại chunk cuối cùng trong buffer
            if current_chunk_buffer:
                processed_document_chunks.append(current_chunk_buffer)
        elif original_text_context.strip():
            processed_document_chunks.append(original_text_context)
        
        chunk_ids_for_this_question = []

        for chunk_text in processed_document_chunks:
            cleaned_chunk = clean_text(chunk_text)
            if not cleaned_chunk:
                continue

            is_duplicate = any(strings_similar(cleaned_chunk, kept_chunk) for kept_chunk in unique_chunks_text)

            if not is_duplicate:
                unique_chunks_text.append(cleaned_chunk)
                
                payload = {
                    "id": global_id_counter,
                    "title": document_title,
                    "article": clean_text(article_info),
                    "context": cleaned_chunk,
                    "passage": cleaned_chunk
                }
                all_payloads.append(payload)
                chunk_ids_for_this_question.append(global_id_counter)
                global_id_counter += 1

        if question and chunk_ids_for_this_question:
            # Lưu label dưới dạng chuỗi JSON
            test_data.append({"question": clean_text(question), "label": json.dumps(chunk_ids_for_this_question)})

    output_jsonl_path = os.path.join(OUTPUT_DIR, "processed_data.jsonl")
    with open(output_jsonl_path, 'w', encoding='utf-8') as f:
        for payload_item in all_payloads:
            f.write(json.dumps(payload_item, ensure_ascii=False) + '\n')
    print(f"✔ Đã lưu {len(all_payloads)} chunks duy nhất vào file: {output_jsonl_path}")

    output_test_csv_path = os.path.join(OUTPUT_DIR, "test.csv")
    if test_data:
        test_df = pd.DataFrame(test_data)
        test_df.to_csv(output_test_csv_path, index=False, encoding='utf-8')
        print(f"✔ Đã tạo và lưu {len(test_df)} mẫu vào file: {output_test_csv_path}")
    else:
        print("🟡 Không có dữ liệu test để tạo file.")
else:
    print("❌ DataFrame rỗng. Bỏ qua giai đoạn xử lý.")

# ==============================================================================
# === 4. TẠO VÀ LƯU EMBEDDINGS (CHẠY CHO TỪNG MODEL) ===========================
# ==============================================================================

print("\nBắt đầu giai đoạn 2: Tạo và lưu embeddings cho từng mô hình...")

if unique_chunks_text:
    for config in MODEL_CONFIGURATIONS:
        model_name = config["name"]
        model_path = config["model_path"]
        output_pkl_path = os.path.join(OUTPUT_DIR, f"embeddings_{model_name}.pkl")
        
        print(f"\n--- Đang xử lý model: {model_name} ---")
        
        try:
            print(f"  - Đang tải model từ: {model_path}...")
            embedding_model = SentenceTransformer(model_path)
            
            print(f"  - Đang tạo embeddings cho {len(unique_chunks_text)} chunks...")
            chunk_embeddings = embedding_model.encode(
                unique_chunks_text, 
                show_progress_bar=True,
                batch_size=32
            )
            
            print(f"  - Đang lưu embeddings vào: {output_pkl_path}...")
            with open(output_pkl_path, 'wb') as f:
                pickle.dump(chunk_embeddings, f)
            print(f"✔ Hoàn tất! Đã lưu {len(chunk_embeddings)} vectors.")

        except Exception as e:
            print(f"❌ Lỗi khi xử lý model {model_name}: {e}")
            continue
else:
    print("🟡 Không có chunk duy nhất nào để tạo embeddings.")

print("\n✨✨✨ HOÀN TẤT TẤT CẢ CÁC GIAI ĐOẠN! ✨✨✨")

2025-06-18 16:51:35.498638: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750265495.517854      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750265495.524716      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Các file output sẽ được lưu tại: /kaggle/working/processed_data_regex

Bắt đầu giai đoạn 1: Chunking và xử lý dữ liệu...
✔ Đã đọc thành công file: /kaggle/input/data-uit/train.csv (7806 hàng)


Đang xử lý các context:   0%|          | 0/7806 [00:00<?, ?it/s]

✔ Đã lưu 478 chunks duy nhất vào file: /kaggle/working/processed_data_regex/processed_data.jsonl
✔ Đã tạo và lưu 292 mẫu vào file: /kaggle/working/processed_data_regex/test.csv

Bắt đầu giai đoạn 2: Tạo và lưu embeddings cho từng mô hình...

--- Đang xử lý model: halong_finetuned ---
  - Đang tải model từ: /kaggle/input/embedding-train/model_test...
  - Đang tạo embeddings cho 478 chunks...


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

...

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

  - Đang tạo embeddings cho 478 chunks...


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

  - Đang lưu embeddings vào: /kaggle/working/processed_data_regex/embeddings_phobert_base.pkl...
✔ Hoàn tất! Đã lưu 478 vectors.

✨✨✨ HOÀN TẤT TẤT CẢ CÁC GIAI ĐOẠN! ✨✨✨
